In [1]:
import cv2
import os
import numpy as np
import glob 

In [2]:
imgdir = "data/vary_pic"#calibration_jpg
for pic in os.listdir(imgdir):
    if pic.endswith(".jpg"):
        img = cv2.imread(os.path.join(imgdir,pic))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = cv2.equalizeHist(gray)
        gray = cv2.resize(gray,(0,0),fx=0.5,fy=0.5)
        #gray = 255*np.power(gray/255,0.5)
        cv2.imwrite("data/new1/"+pic,gray)

In [8]:
img = cv2.imread('data/new1/IMG_20240418_214912.jpg',0)
print(img.shape)
# 确定棋盘格内部角点的数量
patternSize = (7,6)
print("0")
# 在图像中寻找棋盘格角点
ret, corners = cv2.findChessboardCorners(img, patternSize)
print("1")
# 如果找到了角点则绘制它们
print(ret)
if ret == True:
    cv2.namedWindow("video",cv2.WINDOW_NORMAL)#创建一个窗口
    cv2.resizeWindow('video',450,540)#重设大小（只能说normal窗口）
    corners2 = cv2.cornerSubPix(img,corners,(11,11),(-1,-1),(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30,0.001))
    img = cv2.drawChessboardCorners(img, patternSize, corners2,ret)
    cv2.imshow('video',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

(1010, 888)
0
1
True


In [14]:
# 找棋盘格角点
# 阈值
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# 棋盘格模板规格
# 内角点个数，内角点是和其他格子连着的点
w = 6
h = 7
# 世界坐标系中的棋盘格点,例如(0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)，去掉Z坐标，记为二维矩阵
objp = np.zeros((w * h, 3), np.float32)
objp[:, :2] = np.mgrid[0:w, 0:h].T.reshape(-1, 2)
print(objp)
# 储存棋盘格角点的世界坐标和图像坐标对
objpoints = []  # 在世界坐标系中的三维点
imgpoints = []  # 在图像平面的二维点

cv2.namedWindow("findCorners",cv2.WINDOW_NORMAL)#创建一个窗口
cv2.resizeWindow('findCorners',800,800)#重设大小（只能说normal窗口）

images = glob.glob('data/new1/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    print(gray.shape[::-1])
    # 找到棋盘格角点
    # 棋盘图像(8位灰度或彩色图像)  棋盘尺寸  存放角点的位置
    ret, corners = cv2.findChessboardCorners(gray, (w, h), None)
    # 如果找到足够点对，将其存储起来
    if ret == True:
        # 角点精确检测
        # 输入图像 角点初始坐标 搜索窗口为2*winsize+1 死区 求角点的迭代终止条件
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        objpoints.append(objp)
        imgpoints.append(corners)
        #将角点在图像上显示
        cv2.drawChessboardCorners(img, (w, h), corners, ret)
        cv2.imshow('findCorners', img)
        cv2.waitKey(100)
cv2.destroyAllWindows()
# 标定、去畸变
# 输入：世界坐标系里的位置 像素坐标 图像的像素尺寸大小 3*3矩阵，相机内参数矩阵 畸变矩阵
# 输出：标定结果 相机的内参数矩阵 畸变系数 旋转矩阵 平移向量
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
# mtx：内参数矩阵
# dist：畸变系数
# rvecs：旋转向量 （外参数）
# tvecs ：平移向量 （外参数）
print(("ret:"), ret)
print(("mtx:\n"), mtx)  # 内参数矩阵
print(("dist:\n"), dist)  # 畸变系数   distortion cofficients = (k_1,k_2,p_1,p_2,k_3)
print(("rvecs:\n"), rvecs)  # 旋转向量  # 外参数
print(("tvecs:\n"), tvecs)  # 平移向量  # 外参数

[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [5. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [5. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [5. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]
 [5. 3. 0.]
 [0. 4. 0.]
 [1. 4. 0.]
 [2. 4. 0.]
 [3. 4. 0.]
 [4. 4. 0.]
 [5. 4. 0.]
 [0. 5. 0.]
 [1. 5. 0.]
 [2. 5. 0.]
 [3. 5. 0.]
 [4. 5. 0.]
 [5. 5. 0.]
 [0. 6. 0.]
 [1. 6. 0.]
 [2. 6. 0.]
 [3. 6. 0.]
 [4. 6. 0.]
 [5. 6. 0.]]
(2312, 1734)
(2312, 1734)
(1110, 957)
(888, 1010)
(1114, 816)
(1168, 1086)
(931, 912)
(1146, 872)
(1424, 1342)
(1576, 1536)
(1410, 1467)
(1734, 2312)
(1734, 2312)
(1734, 2312)
(1734, 2312)
ret: 3.595562470521688
mtx:
 [[1.93514054e+03 0.00000000e+00 1.02827801e+03]
 [0.00000000e+00 1.94937614e+03 9.16010983e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
 [[ 5.51823137e-02 -1.25741173e+00  1.49114612e-03 -2.98318003e-03
   1.65997587e+00]]
rvecs:
 (array([[-0.11042563],
       [

In [11]:
print(objpoints[0][0].shape)
print(imgpoints[0][0])
print(mtx)
print(dist)
print(len(rvecs))
print(tvecs[0].shape)

(3,)
[[1021.63043  246.13826]]
[[1.93514054e+03 0.00000000e+00 1.02827801e+03]
 [0.00000000e+00 1.94937614e+03 9.16010983e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[ 5.51823137e-02 -1.25741173e+00  1.49114612e-03 -2.98318003e-03
   1.65997587e+00]]
15
(3, 1)


In [18]:
# 已知的真实坐标
real_coordinates = objpoints[-1]

# 已知的相机内参矩阵和外参矩阵
K = mtx
rvec = rvecs[-1]
tvec = tvecs[-1]

for i,real_coordinate in enumerate(real_coordinates):  
    real_coordinate = np.array([real_coordinate])
    #print(real_coordinate)
    # 使用 Rodrigues 函数将旋转向量转换为旋转矩阵
    R, _ = cv2.Rodrigues(rvec)
    
    # 将真实坐标转换为像素坐标（包含畸变）
    #distorted_pixel_coordinates, _ = cv2.projectPoints(real_coordinate[:3].reshape(1, 3), rvec, tvec, K, dist)
    distorted_pixel_coordinates = K@(R@real_coordinate.T+tvec)
    distorted_pixel_coordinates /= distorted_pixel_coordinates[-1]
    distorted_pixel_coordinates = distorted_pixel_coordinates
    distorted_pixel_coordinates = cv2.undistortPoints(distorted_pixel_coordinates[:2], K, dist, P=K)
    imgpoints[-1][i] = distorted_pixel_coordinates[0]
    print("真实坐标:", real_coordinate[:3],"标准像素坐标:",imgpoints[-1][i],"像素坐标（畸变）:", distorted_pixel_coordinates[0])

真实坐标: [[0. 0. 0.]] 标准像素坐标: [[1348.7343 1531.301 ]] 像素坐标（畸变）: [[1348.73429642 1531.30102897]]
真实坐标: [[1. 0. 0.]] 标准像素坐标: [[1182.1562 1530.957 ]] 像素坐标（畸变）: [[1182.15629213 1530.9570371 ]]
真实坐标: [[2. 0. 0.]] 标准像素坐标: [[1014.0358 1532.2865]] 像素坐标（畸变）: [[1014.03583079 1532.28655205]]
真实坐标: [[3. 0. 0.]] 标准像素坐标: [[ 842.97943 1535.1045 ]] 像素坐标（畸变）: [[ 842.97941173 1535.10450383]]
真实坐标: [[4. 0. 0.]] 标准像素坐标: [[ 667.3212 1540.0042]] 像素坐标（畸变）: [[ 667.32119834 1540.00414963]]
真实坐标: [[5. 0. 0.]] 标准像素坐标: [[ 483.9791 1548.4125]] 像素坐标（畸变）: [[ 483.97908443 1548.41252562]]
真实坐标: [[0. 1. 0.]] 标准像素坐标: [[1349.5378 1359.0459]] 像素坐标（畸变）: [[1349.53784323 1359.04589357]]
真实坐标: [[1. 1. 0.]] 标准像素坐标: [[1184.3877 1358.6346]] 像素坐标（畸变）: [[1184.38773348 1358.63461692]]
真实坐标: [[2. 1. 0.]] 标准像素坐标: [[1017.1714 1358.8751]] 像素坐标（畸变）: [[1017.17136839 1358.87514989]]
真实坐标: [[3. 1. 0.]] 标准像素坐标: [[ 847.133  1359.5472]] 像素坐标（畸变）: [[ 847.13301713 1359.54718313]]
真实坐标: [[4. 1. 0.]] 标准像素坐标: [[ 673.34424 1361.0441 ]] 像素坐标（畸变）: [[ 67

In [25]:
# 假设已知的像素坐标
pixel_coordinatess = imgpoints[-1]  # 替换为实际的像素坐标

# 已知的相机内参矩阵和外参矩阵
K = mtx
# 使用 Rodrigues 函数将旋转向量转换为旋转矩阵

rvec = rvecs[-1]
tvec = tvecs[-1]
R, _ = cv2.Rodrigues(rvec)


# 构建投影矩阵 P
P = np.dot(K, R)
for i,pixel_coordinates in enumerate(pixel_coordinatess):
    
    pixel_coordinates = cv2.undistortPoints(pixel_coordinates, K, dist, P=K)
    pixel_coordinates_1 = np.array([np.append(pixel_coordinates,1.0)]).T
    #print(pixel_coordinates_1.shape)

    abc = np.linalg.inv(R)@np.linalg.inv(K)@pixel_coordinates_1
    t = np.linalg.inv(R)@tvec
    s = t[2]/abc[2]
    # 将齐次坐标转换为真实坐标
    real_coordinates_recovered_1 = (s*abc-t).T
    print("标准：",objpoints[0][i],"1未修正畸变:", real_coordinates_recovered_1[:3][0],"相机坐标：",(np.linalg.inv(K)@pixel_coordinates_1)[:,0])

标准： [0. 0. 0.] 1未修正畸变: [-0.05094123 -0.07134021  0.        ] 相机坐标： [0.16773536 0.31876635 1.        ]
标准： [1. 0. 0.] 1未修正畸变: [ 0.98075125 -0.03966579  0.        ] 相机坐标： [0.08031937 0.31720209 1.        ]
标准： [2. 0. 0.] 1未修正畸变: [ 1.99329062 -0.02900279  0.        ] 相机坐标： [-0.00709113  0.31742621  1.        ]
标准： [3. 0. 0.] 1未修正畸变: [ 3.00375413 -0.03590595  0.        ] 相机坐标： [-0.09595944  0.31920375  1.        ]
标准： [4. 0. 0.] 1未修正畸变: [ 4.03098329 -0.06710486  0.        ] 相机坐标： [-0.18803629  0.3232008   1.        ]
标准： [5. 0. 0.] 1未修正畸变: [ 5.11094829 -0.1409125   0.        ] 相机坐标： [-0.28680587  0.33117836  1.        ]
标准： [0. 1. 0.] 1未修正畸变: [-0.01793445  0.98661369  0.        ] 相机坐标： [0.16676304 0.22785009 1.        ]
标准： [1. 1. 0.] 1未修正畸变: [0.99496014 0.99927249 0.        ] 相机坐标： [0.08088634 0.22709038 1.        ]
标准： [2. 1. 0.] 1未修正畸变: [1.99648895 1.00401904 0.        ] 相机坐标： [-0.00558316  0.22700572  1.        ]
标准： [3. 1. 0.] 1未修正畸变: [2.99528689 1.00378462 0.        ] 相机坐标： [-0.09339

In [8]:
# 假设已知的像素坐标
pixel_coordinatess = np.array(points)  # 替换为实际的像素坐标

# 已知的相机内参矩阵和外参矩阵
K = mtx
x1 = 0
y1 = 0
x2 = 0
y2 = 0
for i,pixel_coordinates in enumerate(pixel_coordinatess):

#     pixel_coordinates1 = cv2.undistortPoints(np.float32(pixel_coordinates[0]), K, dist, P=K)
#     pixel_coordinates2 = cv2.undistortPoints(np.float32(pixel_coordinates[1]), K, dist, P=K)
    pixel_coordinates_1 = np.array([np.append(pixel_coordinates[0],1.0)]).T
    pixel_coordinates_2 = np.array([np.append(pixel_coordinates[1],1.0)]).T
    #print(pixel_coordinates_1.shape,pixel_coordinates_2)
    # 使用 Rodrigues 函数将旋转向量转换为旋转矩阵
    
    rvec = rvecs[-4+i]
    tvec = tvecs[-4+i]
    R, _ = cv2.Rodrigues(rvec)
    t = np.linalg.inv(R)@tvec
    
    abc1 = np.linalg.inv(R)@np.linalg.inv(K)@pixel_coordinates_1
    s1 = t[2]/abc1[2]

    abc2 = np.linalg.inv(R)@np.linalg.inv(K)@pixel_coordinates_2
    s2 = t[2]/abc2[2]
    
    # 将齐次坐标转换为真实坐标
    real_coordinates_recovered_1 = (s1*abc1-t).T

    real_coordinates_recovered_2 = (s2*abc2-t).T
    
    x1 += real_coordinates_recovered_1[0][0]
    x2 += real_coordinates_recovered_2[0][0]
    y1 += real_coordinates_recovered_1[0][1]
    y2 += real_coordinates_recovered_2[0][1]
    print("1未修正畸变:", real_coordinates_recovered_1[0])
    print("2未修正畸变:", real_coordinates_recovered_2[0])


NameError: name 'points' is not defined

In [ ]:
print(x1/4,y1/4,x2/4,y2/4)
area = (x2/4-x1/4)*(y2/4-y1/4)*0.08*0.08#*np.pi/4
print(abs(area))

In [ ]:
img0 = cv2.imread("data\\template\\fang.jpg")
# img0 = cv2.imread("data/template/yuan.jpg")
sift = cv2.SIFT_create()  # 使用SIFT特征提取器
# kp0, des0 = sift.detectAndCompute(img0, None)
kp0, des0 = sift.detectAndCompute(img0, None)

In [ ]:
pics = 'data/new'
points = []
for pic in os.listdir(pics):
    if pic.endswith('.jpg'):
        # 定义FLANN匹配器------KD树
        img = cv2.imread(os.path.join(pics,pic))
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
    
        kp, des = sift.detectAndCompute(img, None)
    
        # 匹配图像
        matches = flann.knnMatch(des0, des, k=2)
        good = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)
    
        if len(good) > 7:
            src_pts = np.float32([kp0[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
            matchesMask = mask.ravel().tolist()
            h, w, dim = img0.shape
            pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
            dst = cv2.perspectiveTransform(pts, M)
            dst = np.int32(dst)
            x1, y1 = dst[0][0]
            x2, y2 = dst[2][0]
            points.append([[x1,y1],[x2,y2]])
            img = cv2.polylines(img, [np.int32(dst)], True, (255, 255, 255), 3, cv2.LINE_AA)
            draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
            img2 = cv2.drawMatches(img0, kp0, img, kp, good, None, **draw_params)
            cv2.imwrite(os.path.join("data/out", pic), img2)

In [ ]:
print(points[0])